In [64]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

#### Problem 1

In [65]:
x = 0.65

In [66]:
# Define the function and its derivative
def f(x):
    return np.exp(-x / 4)

def f_prime(x):
    return (-1/4) * np.exp(-x/4)


In [67]:
h4 = 10**(-4)
h5 = 10**(-5)
h6 = 10**(-6)
h7 = 10**(-7)

def f_center(x,h):
    return (f(x+h) - f(x-h)) / (2*h)

def frac_error(a,t):
    return np.abs(1-(a/t))

true_deriv = f_prime(x)

print("The derivative with step size 10^-4 is {} and the fractional error is {}".format(f_center(x,h4), frac_error(f_center(x,h4),true_deriv)))
print("The derivative with step size 10^-5 is {} and the fractional error is {}".format(f_center(x,h5), frac_error(f_center(x,h5),true_deriv)))
print("The derivative with step size 10^-6 is {} and the fractional error is {}".format(f_center(x,h6), frac_error(f_center(x,h6),true_deriv)))
print("The derivative with step size 10^-7 is {} and the fractional error is {}".format(f_center(x,h7), frac_error(f_center(x,h7),true_deriv)))

The derivative with step size 10^-4 is -0.2125040225781083 and the fractional error is 1.0239231684749939e-10
The derivative with step size 10^-5 is -0.21250402255645892 and the fractional error is 5.146993942162226e-13
The derivative with step size 10^-6 is -0.21250402260086787 and the fractional error is 2.0949397772085376e-10
The derivative with step size 10^-7 is -0.21250402271189017 and the fractional error is 7.319420625151452e-10


#### Problem 2

In [68]:
def richardson_center (f, z, h, nsteps, args=()):
    """Evaluate the first derivative of a function at z, that is f'(z),
    using Richardson extrapolation and center differencing.

    Returned is the full table of approximations, Fij for j <= i. The
    values of Fij for j > i are set to zero. The final value F[-1,-1]
    should be the most accurate estimate.

    Parameters
    ----------
    f : function
        Vectorized Python function.
        This is the function for which we are estimating the derivative.
    z : number
        Value at which to evaluate the derivative.
    h : number
        Initial stepsize.
    nsteps : integer
        Number of steps to perform.
    args : tuple, optional
        extra arguments to pass to the function, f.
    """
    # Extra check to allow for args=(1) to be handled properly. This is a
    # technical detail that you do not need to worry about.
    if not isinstance(args, (tuple, list, np.ndarray)):
        args = (args,)
    # Create a zero filled table for our estimates
    F = np.zeros((nsteps, nsteps))
    # First column of F is the center differencing estimate.
    # We can fill this without a loop!
    harr = h / 2.**np.arange(nsteps)
    F[:,0] = (f(z+harr, *args) - f(z-harr, *args)) / (2.*harr)
    # Now iterate, unfortunately we do need one loop. We could
    # get rid of the inner loop but the algorithm is a little easier to
    # understand if we do not.
    for i in range(1, nsteps):
        fact = 0.25
        for j in range(1, i+1):
            F[i,j] = F[i-1,j-1] - (F[i-1,j-1] - F[i,j-1])/ (1-fact)
            fact *= 0.25
    return F

In [69]:
def richardson_forward(f, z, h, nsteps, args=()):
    """Evaluate the first derivative of a function at z, that is f'(z),
    using Richardson extrapolation and forward differencing.

    Returned is the full table of approximations, Fij for j <= i. The
    values of Fij for j > i are set to zero. The final value F[-1,-1]
    should be the most accurate estimate.

    Parameters
    ----------
    f : function
        Vectorized Python function.
        This is the function for which we are estimating the derivative.
    z : number
        Value at which to evaluate the derivative.
    h : number
        Initial stepsize.
    nsteps : integer
        Number of steps to perform.
    args : tuple, optional
        extra arguments to pass to the function, f.
    """
    # Extra check to allow for args=(1) to be handled properly. This is a
    # technical detail that you do not need to worry about.
    if not isinstance(args, (tuple, list, np.ndarray)):
        args = (args,)
    # Create a zero filled table for our estimates
    F = np.zeros((nsteps, nsteps))
    # First column of F is the center differencing estimate.
    # We can fill this without a loop!
    harr = h / 2.**np.arange(nsteps)
    F[:,0] = (f(z+harr, *args) - f(z, *args)) / harr
    # Now iterate, unfortunately we do need one loop. We could
    # get rid of the inner loop but the algorithm is a little easier to
    # understand if we do not.
    for i in range(1, nsteps):
        q = 2
        for j in range(1, i+1):
            F[i,j] = (q*F[i,j-1] - F[i-1,j-1])/ (q-1)
            q *= 2
    return F

In [70]:
def f(x):
    return (3**x) * np.sin(x)

def f_prime(x):
    return (3**x) * np.cos(x) + (np.log(3) * 3**x * np.sin(x))

true_deriv = f_prime(1.15)

richardson_center_answer = richardson_center(f, 1.15, 0.4, 8)
richardson_forward_answer = richardson_forward(f, 1.15, 0.4, 8)

print("Richardson_center: {}. Frac Error: {}".format(richardson_center_answer[-1,-1], frac_error(richardson_center_answer[-1,-1],true_deriv)))
print("Richardson_forward: {}. Frac Error: {}".format(richardson_forward_answer[-1,-1], frac_error(richardson_forward_answer[-1,-1],true_deriv)))

Richardson_center: 4.992255689431632. Frac Error: 1.4432899320127035e-14
Richardson_forward: 4.9922556894312375. Frac Error: 6.461498003318411e-14


In [75]:
richardson_center_answer = richardson_center(f, 1.15, 0.4, 6)
richardson_forward_answer = richardson_forward(f, 1.15, 0.4, 9)

print("Richardson_center: {}. Frac Error: {}".format(richardson_center_answer[-1,-1], frac_error(richardson_center_answer[-1,-1],true_deriv)))
print("Richardson_forward: {}. Frac Error: {}".format(richardson_forward_answer[-1,-1], frac_error(richardson_forward_answer[-1,-1],true_deriv)))

Richardson_center: 4.992255689431567. Frac Error: 1.3322676295501878e-15
Richardson_forward: 4.992255689431513. Frac Error: 9.43689570931383e-15
